# Ranking documents using skip grams word embedding model for phrase queries

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import *
from nltk.stem.porter import *
import pickle
import gensim 
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [4]:
# ! Uncomment in first run 
# nltk.download('punkt')
# nltk.download('words')
# nltk.download('stopwords')

## Setup
#### The corpus/documents are extracted from the pickle files.
#### Stemmer has been initialised to convert term to its root form (Example : received -> receive)

In [5]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
root = Path("../")

my_path = root / "Pickled_files" / "Documents"
dbfile = open(my_path, 'rb')     
documents = pickle.load(dbfile)
dbfile.close()

## Model

Skip-gram model is a type of neural network architecture that is commonly used for word embedding. It is a form of unsupervised learning that aims to learn the distributional representation of words.The basic idea behind skip-gram model is to predict the context words surrounding a given target word, rather than predicting the target word from the context words. The context words are defined as the words that occur within a certain window of the target word.

The skip-gram model consists of an input layer, a hidden layer, and an output layer. The input layer represents the target word, and the output layer represents the context words. The hidden layer is used to transform the input word into a distributed representation, or embedding, which is used to predict the context words.During training, the skip-gram model is fed with a large corpus of text. For each target word in the corpus, a training example is created by randomly selecting one of its context words as the output, and setting the remaining context words as inputs to the model. The model is then trained to predict the output context word given the input context words.

The training process involves adjusting the weights of the model to minimize the difference between the predicted context word and the actual context word. The weights are adjusted using backpropagation algorithm, which updates the weights based on the difference between the predicted and actual output.After training, the skip-gram model generates a dense vector representation for each word in the vocabulary. The vector representation captures the semantic and syntactic information of the word, and can be used as input to other machine learning models for various natural language processing tasks such as text classification, information retrieval, and machine translation.

## Building the model

The following code initializes an empty list called data, which will be used to store the tokenized and preprocessed documents. Then, it loops over each document in the documents list and tokenizes each sentence into a list of words. Each word is then converted to lowercase and added to a temporary list called temp. The temp list is then appended to the data list, which now contains a list of tokenized and preprocessed documents.

Next, the Word2Vec model from the gensim library is used to train a word embedding model on the data list. The min_count parameter specifies the minimum frequency count of a word in the corpus for it to be included in the model. The vector_size parameter specifies the dimensionality of the word vectors. The window parameter specifies the maximum distance between the target word and the context word within a sentence. The sg parameter specifies the training algorithm to be used - 1 for skip-gram and 0 for CBOW. In this case, sg is set to 1 which corresponds to the skip-gram algorithm for training.

After training, the word embedding model skipgram_model contains dense vector representations for each word in the corpus. These vector representations can be used for various natural language processing tasks such as text classification, information retrieval, and machine translation.

In [6]:
data = list()
for i in documents: 
    temp = list()
    # tokenize the sentence into words 
    for j in word_tokenize(i[0]): 
        temp.append(j.lower()) 
    data.append(temp)

skipgram_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5,sg = 1)

## Generating word embeddings for documents

The following code defines two functions.

1.**preprocess(text)** takes a string of text as input, tokenizes it, removes stop words and filters out words with length less than two. It returns a list of preprocessed tokens.

2.**text_embedding(text, model)** takes a string of text and a pre-trained word embedding model as inputs. It preprocesses the text using the preprocess() function, obtains the word embeddings for each word in the preprocessed text from the pre-trained word embedding model, takes the average of the word embeddings to generate a document embedding, and returns the document embedding as a numpy array.

Finally, the **document_vecs** variable is assigned a list of document embeddings generated using the text_embedding() function and the pre-trained word embedding model (skipgram_model) for each document in the documents list.


In [7]:
def preprocess(text):
    tokens = []
    for word in word_tokenize(text, language='english'):
        if len(word) >= 2 and word not in stop_words:
            tokens.append(word)
    return tokens

# Define a function to generate text embeddings
def text_embedding(text, model):
    # Preprocess the text (tokenize, remove stop words, stem)
    preprocessed_text = preprocess(text)
    # Get the word embeddings for each word in the document
    word_embeddings = [model.wv[word] for word in preprocessed_text if word in model.wv.key_to_index]
    # Take the average of the word embeddings to generate a document embedding
    if len(word_embeddings) > 0:
        document_embedding = np.mean(word_embeddings, axis=0)
    else:
        document_embedding = np.zeros(model.vector_size)
    return document_embedding

document_vecs = [text_embedding(document[0], skipgram_model) for document in documents]

In [8]:
pd.DataFrame(document_vecs)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.004590,0.031992,0.149353,-0.103647,0.127495,-0.345164,0.255643,0.259811,0.157978,-0.124985,...,0.450639,0.028335,0.206324,-0.063071,0.110578,0.085154,0.278110,-0.201369,0.117699,0.060710
1,0.105189,0.132897,0.044019,-0.042159,0.179388,-0.327574,0.260852,0.266546,0.189100,-0.069553,...,0.422972,0.107587,0.161347,-0.126538,0.073219,0.122300,0.380425,0.048626,0.157927,-0.044815
2,0.131420,0.056091,0.078574,-0.137063,0.160461,-0.292200,0.185550,0.263968,0.205161,-0.011723,...,0.424150,0.119437,0.203855,-0.044585,0.162057,0.188318,0.476668,0.031398,0.156915,0.177543
3,0.133949,0.110150,0.059820,-0.149727,0.158738,-0.300730,0.271797,0.353809,0.208494,-0.062116,...,0.501836,0.067628,0.204282,-0.040796,0.039807,0.139566,0.335987,-0.004681,0.147508,0.228402
4,0.089234,0.104596,0.083269,0.031202,0.128677,-0.220117,0.220752,0.196400,0.252232,-0.092232,...,0.421696,0.113761,0.252042,-0.097961,0.155823,0.296137,0.377603,-0.072417,0.211872,0.034544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3470,-0.059292,0.093661,-0.053745,-0.151095,0.096664,-0.395085,0.341605,0.403616,0.341046,-0.082672,...,0.528257,0.013550,0.255730,-0.059689,0.189930,0.266602,0.204304,-0.140718,0.181028,0.086290
3471,-0.030575,0.071396,0.085975,-0.094965,0.092630,-0.230809,0.297596,0.284864,0.196800,-0.108274,...,0.421334,0.011176,0.259359,-0.111183,0.146449,0.164449,0.301710,-0.153121,0.089725,0.024233
3472,-0.099716,0.155442,0.045003,0.054721,0.055121,-0.204208,0.335802,0.325876,0.011340,-0.106860,...,0.353255,-0.073040,0.146084,-0.139748,0.175605,0.189560,0.096747,-0.423187,0.084722,0.026079
3473,-0.182390,0.117490,0.040536,0.028133,0.118737,-0.290774,0.430598,0.375839,-0.037987,-0.235611,...,0.438862,-0.150765,0.165542,-0.209222,0.242864,0.189597,0.071091,-0.648700,-0.017442,0.066832


## Similarity measure

Here cosine_similarity is used to compare the query vector and and the document vector.

In [9]:
# Define a function to calculate cosine similarity between two vectors
def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

## Saving document vectors for future use

In [10]:
my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'wb')
pickle.dump(document_vecs, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'rb')     
document_vecs = pickle.load(dbfile)
dbfile.close()

## Processing the query

This code calculates the similarity scores between a given query and a set of documents. It first generates a document vector for each document in the set using the text_embedding function. Then, it generates a query vector using the same function. The cosine similarity between the query vector and each document vector is calculated and stored in the similarity_scores list. Finally, the documents are ranked in descending order based on their similarity scores with the query, and the top five documents are printed.

In [11]:
def printResults(query,ranked_docs):
    print(query)
    print()
    for i in range(0,min(3,len(ranked_docs))):
        print(ranked_docs[i][1])
        print(ranked_docs[i][0])
        print("---------------------------------------------------------------------------------------------------------------")

In [13]:
# Example usage: compare a query with a set of documents
input_file = open("phrase_input.txt","r")
query = input_file.readline()
query_vec = text_embedding(query,skipgram_model)
# Calculate similarity scores between the query and all documents
similarity_scores = [cosine_similarity(query_vec, document_vec) for document_vec in document_vecs]
# Rank documents based on similarity scores
ranked_documents = [document for _, document in sorted(zip(similarity_scores, documents), reverse=True)]
# Print the ranked documents
printResults(query,ranked_documents)

Medical reports

.\Docs\Auto\AU127-1.docx
proof of claim ; medical reports as soon as possible , any person making claim must give us written proof of claim.the injured person may be required to take medical examinations by physicians we choose , as often as we reasonably require.we must be given authorization to obtain medical reports and copies of records pertinent to the claim . 
---------------------------------------------------------------------------------------------------------------
.\Docs\Property\eSols Property Owners Commercial Policy Wording 2018 Policies which incepted before 01 July.docx
we will only invoke this right in exceptional circumstances as a result of you behaving inappropriately , for example : where we have a reasonable suspicion of fraud you use threatening or abusive behaviour or language or intimidation or bullying of our staff or suppliers where it is found that you , deliberately or recklessly , disclosed false information or failed to disclose importan